In [8]:
from graphics import *
import numpy as np

In [16]:
def main(list_point, sudut):
    """
    PROGRAM UTAMA
    Fungsi untuk menampilkan pemutaran balok
    
    Args: 
        list_point (list) : List titik sudut bangun ruang
        sudut (int) : Seberapa besar bangun ruang tersebut akan diputar (satuan: derajat)
        
    Returns:
        None
    """
    win = GraphWin('Awal', 200,200)
    win.setCoords(-3,-3,3,3)
    
#     sumbu = Line(Point(-3,-3),Point(3,3))
#     sumbu.draw(win)
        
    for iterr in range(sudut):
        isFinished = True if (iterr==sudut-1) else False        
        draw(win, list_point, isFinished)
        list_point = Rotate(list_point)
    
    win.getMouse()
    win.close()

In [17]:
def draw(windows, list_point, finished):
    """
    Fungsi untuk Menggambar titik sudut dan garis kerangka balok
    
    Args: 
        windows (graphics.GraphWin): window dimana objek akan digambar
        list_point (list) : list titik sudut yang akan digambar
        finished (bool) : flag apakah iterasi ini adalah iterasi terakhir
        
    Returns:
        None
    """
    
    points = []
    lines = []
    for i, koor_point in enumerate(list_point):
        point_temp = Point(koor_point[1],koor_point[2])
        points.append(point_temp)
        points[i].draw(windows)
    
    # Menggambar titik sudut kubus
    for j in range(0,5,4):
        for k in range(j, 4+j):
            line_temp = None
            if (k+1) != 4+j:
                line_temp = Line(points[k], points[k+1])
            else:
                line_temp = Line(points[k], points[k-3])
            lines.append(line_temp)
            
            lines[k].draw(windows)
            
    for l in range(4):
        line_temp = Line(points[l],points[l+4])
        lines.append(line_temp)
        lines[8+l].draw(windows)
    
    time.sleep(.07)
    
    if not finished:
        for point in points:
            point.undraw()
        for line in lines:
            line.undraw()

In [18]:
def Rotate(list_point):
    """
    Fungsi untuk merotasi titik
    
    Args:
        list_point (list) : list titik yang akan dirotasi
    
    Returns:
        List titik setelah dirotasi
    """
    list_point_transpos = np.array(list_point).T
    
    # Rotasi 45 derajat thd sumbu 3,3,3 - 5,5,5 (sesuai PR kemarin)
    # Rumus: 
    # cos dan sin (-45)
    cos_45 = np.cos(np.radians(-45))
    sin_45 = np.sin(np.radians(-45))

    # miu (sbg argumen matriks R(miu)), 
    miu_rad = np.arctan(1/np.sqrt(2))
    miu = np.degrees(miu_rad)

    cos45 = np.cos(np.radians(45))
    sin45 = np.sin(np.radians(45))

    # Matriks translasi 3,3,3-5,5,5 ke origin
    T = np.array([[1,0,0,-3],
                  [0,1,0,-3],
                  [0,0,1,-3],
                  [0,0,0,1]])

    # matriks untuk merotasi sumbu putar (3,3,3-5,5,5) sebesar -45 derajat ke bidang yz
    R_45 = np.array([[cos_45, 0, sin_45, 0],
                  [0,      1, 0,      0],
                  [-sin_45,0, cos_45, 0],
                  [0,      0, 0,      1]])
    
    # Matriks utk merotasi sumbu putar ke sumbu z
    Rmiu = np.array([[1,0,0,0],
                     [0,np.cos(miu_rad), -np.sin(miu_rad), 0],
                     [0,np.sin(miu_rad), np.cos(miu_rad), 0],
                     [0,0,0,1]])
    
    # Rotasi sebesar sudut yg diinginkan
    Rotasi = np.array([[np.cos(np.radians(1)), -np.sin(np.radians(1)),0,0],
                       [np.sin(np.radians(1)), np.cos(np.radians(1)), 0,0],
                       [0,                     0,                     1,0],
                       [0,                     0,                     0,1]])
    
    # Inverse Rmiu
    R_miu = np.array([[1,0,0,0],
                     [0,np.cos(-miu_rad), -np.sin(-miu_rad), 0],
                     [0,np.sin(-miu_rad), np.cos(-miu_rad), 0],
                     [0,0,0,1]])
    
    # Inverse R_45
    R45 = np.array([[cos45, 0, sin45, 0],
                  [0,      1, 0,      0],
                  [-sin45,0, cos45, 0],
                  [0,      0, 0,      1]])
    
    # Inverse T
    T_1 = np.linalg.inv(T)

    # Matriks rotasi keseluruhan
    M = T_1 @ R45 @R_miu @ Rotasi @Rmiu @ R_45 @ T
    
    return (M @ list_point_transpos).T

In [ ]:
# Setup koordinat balok
A = np.array([1,-1,1,1])
B = np.array([1,1,1,1])
C = np.array([-1,1,1,1])
D = np.array([-1,-1,1,1])
E = np.array([1,-1,-1,1])
F = np.array([1,1,-1,1])
G = np.array([-1,1,-1,1])
H = np.array([-1,-1,-1,1])

# Matriks seluruh koordinat balok
points_awal = [A,B,C,D,E,F,G,H]

# Run program
main(points_awal, sudut=360)